In [1]:
#查询下GPU的数量已经当下的情况
import torch

# 查询GPU数量
print(f"当前GPU数量: {torch.cuda.device_count()}")


当前GPU数量: 4


In [2]:
# 查询当前GPU使用情况
# Print GPU usage information
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"设备 {i}:")
        print(f" - 名称: {torch.cuda.get_device_name(i)}")
        print(f" - 使用情况: {torch.cuda.memory_allocated(i) / (1024 ** 3):.2f} GB 已分配 / {torch.cuda.memory_reserved(i) / (1024 ** 3):.2f} GB 被预留")
else:
    print("没有可用的GPU。")



设备 0:
 - 名称: NVIDIA A100-SXM4-80GB
 - 使用情况: 0.00 GB 已分配 / 0.00 GB 被预留
设备 1:
 - 名称: NVIDIA A100-SXM4-80GB
 - 使用情况: 0.00 GB 已分配 / 0.00 GB 被预留
设备 2:
 - 名称: NVIDIA A100-SXM4-80GB
 - 使用情况: 0.00 GB 已分配 / 0.00 GB 被预留
设备 3:
 - 名称: NVIDIA A100-SXM4-80GB
 - 使用情况: 0.00 GB 已分配 / 0.00 GB 被预留


In [3]:
# 查看PyTorch和CUDA版本
import torch

print("=" * 60)
print("PyTorch 和 CUDA 版本信息")
print("=" * 60)
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 是否可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"cuDNN 版本: {torch.backends.cudnn.version()}")
    print(f"cuDNN 是否启用: {torch.backends.cudnn.enabled}")
    print(f"\nGPU 设备信息:")
    for i in range(torch.cuda.device_count()):
        print(f"  设备 {i}: {torch.cuda.get_device_name(i)}")
        print(f"    CUDA 计算能力: {torch.cuda.get_device_capability(i)}")
        print(f"    总显存: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB")
else:
    print("CUDA 不可用，请检查CUDA安装")

print("=" * 60)


PyTorch 和 CUDA 版本信息
PyTorch 版本: 2.9.1+cu128
CUDA 是否可用: True
CUDA 版本: 12.8
cuDNN 版本: 91300
cuDNN 是否启用: True

GPU 设备信息:
  设备 0: NVIDIA A100-SXM4-80GB
    CUDA 计算能力: (8, 0)
    总显存: 79.44 GB
  设备 1: NVIDIA A100-SXM4-80GB
    CUDA 计算能力: (8, 0)
    总显存: 79.44 GB
  设备 2: NVIDIA A100-SXM4-80GB
    CUDA 计算能力: (8, 0)
    总显存: 79.44 GB
  设备 3: NVIDIA A100-SXM4-80GB
    CUDA 计算能力: (8, 0)
    总显存: 79.44 GB


In [4]:
# 查看CPU型号和详细信息
import platform
import os

print("=" * 60)
print("CPU 信息")
print("=" * 60)

# 方法1: 使用platform模块
print(f"处理器: {platform.processor()}")
print(f"架构: {platform.machine()}")
print(f"平台: {platform.platform()}")

# 方法2: 读取/proc/cpuinfo获取详细信息（Linux系统）
if os.path.exists('/proc/cpuinfo'):
    print("\n详细CPU信息:")
    with open('/proc/cpuinfo', 'r') as f:
        cpuinfo = f.read()
        # 提取CPU型号
        for line in cpuinfo.split('\n'):
            if 'model name' in line.lower():
                print(f"  {line.strip()}")
                break
        # 提取CPU核心数
        for line in cpuinfo.split('\n'):
            if 'cpu cores' in line.lower():
                print(f"  {line.strip()}")
                break
        # 提取物理CPU数量
        physical_cpus = set()
        for line in cpuinfo.split('\n'):
            if 'physical id' in line.lower():
                physical_cpus.add(line.strip())
        if physical_cpus:
            print(f"  物理CPU数量: {len(physical_cpus)}")
        # 提取逻辑核心数
        logical_cores = len([line for line in cpuinfo.split('\n') if line.startswith('processor')])
        print(f"  逻辑核心数: {logical_cores}")
        # 提取CPU频率
        for line in cpuinfo.split('\n'):
            if 'cpu mhz' in line.lower() or 'bogomips' in line.lower():
                print(f"  {line.strip()}")
                break

# 方法3: 使用os.cpu_count()
print(f"\nPython检测到的CPU核心数: {os.cpu_count()}")

print("=" * 60)


CPU 信息
处理器: x86_64
架构: x86_64
平台: Linux-6.8.0-87-generic-x86_64-with-glibc2.35

详细CPU信息:
  model name	: Intel(R) Xeon(R) CPU E5-2699C v4 @ 2.20GHz
  cpu cores	: 22
  物理CPU数量: 2
  逻辑核心数: 88
  cpu MHz		: 1200.000

Python检测到的CPU核心数: 88


In [ ]:
# 修复bug后的GPU张量核心加速算力测试
import time
import torch
import os

# 在PyTorch中启用TF32（可能默认已开启，建议确认）
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

def get_theoretical_flops(gpu_name):
    """
    根据GPU型号返回理论算力值
    """
    gpu_name_upper = gpu_name.upper()
    # A100系列理论值
    if 'A100' in gpu_name_upper:
        return {
            'FP32': 19.5,       # TFLOPS (不使用张量核心，标准FP32)
            'TF32': 156.0,      # TFLOPS (使用张量核心)
            'FP16': 312.0,      # TFLOPS
            'BFLOAT16': 312.0,  # TFLOPS
            'INT8': 624.0       # TOPS
        }
    # H100系列理论值
    elif 'H100' in gpu_name_upper:
        return {
            'FP32': 67.0,       # TFLOPS (不使用张量核心，标准FP32)
            'TF32': 1000.0,     # TFLOPS (使用张量核心)
            'FP16': 2000.0,     # TFLOPS
            'BFLOAT16': 2000.0, # TFLOPS
            'INT8': 4000.0      # TOPS
        }
    # V100系列理论值
    elif 'V100' in gpu_name_upper:
        return {
            'FP32': 15.7,       # TFLOPS (不使用张量核心，标准FP32)
            'TF32': 0.0,        # V100不支持TF32
            'FP16': 125.0,      # TFLOPS
            'BFLOAT16': 0.0,    # V100不支持BFLOAT16
            'INT8': 0.0         # V100不支持INT8张量核心
        }
    # RTX 3090/4090等消费级GPU
    elif 'RTX 3090' in gpu_name_upper or 'RTX 4090' in gpu_name_upper:
        if '4090' in gpu_name_upper:
            return {
                'FP32': 83.0,       # TFLOPS (不使用张量核心，标准FP32)
                'TF32': 0.0,        # RTX 4090不支持TF32
                'FP16': 165.0,      # TFLOPS (约)
                'BFLOAT16': 165.0,  # TFLOPS (约)
                'INT8': 1320.0      # TOPS (约)
            }
        else:  # RTX 3090
            return {
                'FP32': 36.0,       # TFLOPS (不使用张量核心，标准FP32)
                'TF32': 0.0,        # RTX 3090不支持TF32
                'FP16': 142.0,      # TFLOPS (约)
                'BFLOAT16': 142.0,  # TFLOPS (约)
                'INT8': 1140.0      # TOPS (约)
            }
    # 默认值（未知GPU型号）
    else:
        return {
            'FP32': 0.0,
            'TF32': 0.0,
            'FP16': 0.0,
            'BFLOAT16': 0.0,
            'INT8': 0.0
        }

def benchmark_gpu_flops(device_id, dtype, warmup_iterations=10, test_iterations=100, size=4096):
    """
    测试GPU在不同精度下的实际算力

    参数:
        device_id: GPU设备ID
        dtype: 数据类型 (torch.float32, torch.float16, torch.bfloat16)
        warmup_iterations: 预热迭代次数
        test_iterations: 测试迭代次数
        size: square matrix size，用于性能对比，默认4096
    返回:
        实际测得的TFLOPS/TOPS
    """
    device = torch.device(f'cuda:{device_id}')
    torch.cuda.set_device(device)
    # 构造两个矩阵
    if dtype == torch.int8:
        # INT8不同测试方式，此处暂不支持如需求再添加
        raise NotImplementedError('INT8暂未实现基准测试')
    else:
        a = torch.randn(size, size, device=device, dtype=dtype)
        b = torch.randn(size, size, device=device, dtype=dtype)

    # 预热
    torch.cuda.synchronize(device)
    for _ in range(warmup_iterations):
        c = torch.matmul(a, b)
    torch.cuda.synchronize(device)

    # 正式测试
    torch.cuda.synchronize(device)
    start_time = time.time()
    for _ in range(test_iterations):
        c = torch.matmul(a, b)
    torch.cuda.synchronize(device)
    end_time = time.time()

    # FLOP计算公式，矩阵乘法: 2*N^3每一次
    total_ops = 2 * (size ** 3) * test_iterations
    elapsed_time = end_time - start_time

    tflops = (total_ops / elapsed_time) / 1e12
    return tflops

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"当前GPU: {gpu_name}")
        #用来过滤卡的工作
        # if i == 0 or i == 2 or i == 3:
        #     continue
        theoretical = get_theoretical_flops(gpu_name)
        print(f"\n设备 {i} ({gpu_name}) 的张量核心加速能力:")
        print("-" * 80)
        print(f"{'精度':<12} {'实际值':<15} {'理论值':<15} {'达成率':<12} {'状态':<10}")
        print("-" * 80)
        # 测试FP32 (标准单精度，不使用张量核心)
        try:
            torch.backends.cuda.matmul.allow_tf32 = False
            torch.backends.cudnn.allow_tf32 = False
            tflops_fp32 = benchmark_gpu_flops(i, torch.float32)
            if theoretical['FP32'] > 0:
                ratio = (tflops_fp32 / theoretical['FP32']) * 100
                status = "✓ 正常" if ratio >= 50 else "⚠ 偏低"
                print(f"FP32       {tflops_fp32:>8.2f} TFLOPS  {theoretical['FP32']:>8.2f} TFLOPS  {ratio:>7.2f}%    {status}")
            else:
                print(f"FP32       {tflops_fp32:>8.2f} TFLOPS  {'不支持':<15} {'N/A':<12} {'-':<10}")
        except Exception as e:
            print(f"FP32       测试失败 ({e})")

        # 测试TF32 (需torch>=1.7; 通过torch.float32, PyTorch会自动用TF32)
        try:
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            tflops_tf32 = benchmark_gpu_flops(i, torch.float32)
            if theoretical['TF32'] > 0:
                ratio = (tflops_tf32 / theoretical['TF32']) * 100
                status = "✓ 正常" if ratio >= 50 else "⚠ 偏低"
                print(f"TF32       {tflops_tf32:>8.2f} TFLOPS  {theoretical['TF32']:>8.2f} TFLOPS  {ratio:>7.2f}%    {status}")
            else:
                print(f"TF32       {tflops_tf32:>8.2f} TFLOPS  {'不支持':<15} {'N/A':<12} {'-':<10}")
        except Exception as e:
            print(f"TF32       测试失败 ({e})")
        # 测试FP16
        try:
            tflops_fp16 = benchmark_gpu_flops(i, torch.float16)
            if theoretical['FP16'] > 0:
                ratio = (tflops_fp16 / theoretical['FP16']) * 100
                status = "✓ 正常" if ratio >= 50 else "⚠ 偏低"
                print(f"FP16       {tflops_fp16:>8.2f} TFLOPS  {theoretical['FP16']:>8.2f} TFLOPS  {ratio:>7.2f}%    {status}")
            else:
                print(f"FP16       {tflops_fp16:>8.2f} TFLOPS  {'不支持':<15} {'N/A':<12} {'-':<10}")
        except Exception as e:
            print(f"FP16       测试失败 ({e})")

        # 测试BFLOAT16
        try:
            tflops_bf16 = benchmark_gpu_flops(i, torch.bfloat16)
            if theoretical['BFLOAT16'] > 0:
                ratio = (tflops_bf16 / theoretical['BFLOAT16']) * 100
                status = "✓ 正常" if ratio >= 50 else "⚠ 偏低"
                print(f"BFLOAT16   {tflops_bf16:>8.2f} TFLOPS  {theoretical['BFLOAT16']:>8.2f} TFLOPS  {ratio:>7.2f}%    {status}")
            else:
                print(f"BFLOAT16   {tflops_bf16:>8.2f} TFLOPS  {'不支持':<15} {'N/A':<12} {'-':<10}")
        except Exception as e:
            print(f"BFLOAT16   测试失败 ({e})")

     
        print("📝 说明:")
        print("   • 达成率基于理论峰值算力，PyTorch实际性能通常在理论值的60-85%范围内是正常的")
        print("   • FP32: 标准单精度浮点数，不使用张量核心，性能较低但精度最高")
        print("   • TF32: 使用张量核心加速的FP32变体，性能比FP32高约8倍")
        print("   • 📚 详细说明请查看下面的Markdown单元格")
        print()
else:
    print("没有可用的GPU。")


当前GPU: NVIDIA A100-SXM4-80GB
当前GPU: NVIDIA A100-SXM4-80GB

设备 1 (NVIDIA A100-SXM4-80GB) 的张量核心加速能力:
--------------------------------------------------------------------------------
精度           实际值             理论值             达成率          状态        
--------------------------------------------------------------------------------
FP32          17.39 TFLOPS     19.50 TFLOPS    89.18%    ✓ 正常
TF32          98.48 TFLOPS    156.00 TFLOPS    63.13%    ✓ 正常
FP16         245.83 TFLOPS    312.00 TFLOPS    78.79%    ✓ 正常
BFLOAT16     229.10 TFLOPS    312.00 TFLOPS    73.43%    ✓ 正常
📝 说明:
   • 达成率基于理论峰值算力，PyTorch实际性能通常在理论值的60-85%范围内是正常的
   • FP32: 标准单精度浮点数，不使用张量核心，性能较低但精度最高
   • TF32: 使用张量核心加速的FP32变体，性能比FP32高约8倍
   • 📚 详细说明请查看下面的Markdown单元格

当前GPU: NVIDIA A100-SXM4-80GB
当前GPU: NVIDIA A100-SXM4-80GB


In [5]:
import torch
import time

def test_inter_gpu_bandwidth(size_gb=1, num_iters=10):
    """测试多张GPU之间的带宽和延迟
    Args:
        size_gb (int): 每次拷贝的数据量，单位GB
        num_iters (int): 拷贝次数
    """
    if torch.cuda.device_count() < 2:
        print("需要至少两张GPU进行测试。")
        return

    src = 0
    dst = 1
    size = int(size_gb * 1024**3 // 4)  # float32, 4B
    print(f"\n{'='*80}")
    print(f"测试跨GPU拷贝速度: 从GPU{src} --> GPU{dst} ({size_gb} GB, {num_iters}次)")

    # 创建张量
    tensor_src = torch.randn(size, dtype=torch.float32, device=f'cuda:{src}')
    torch.cuda.synchronize()
    times = []
    for _ in range(num_iters):
        torch.cuda.synchronize()
        start = time.time()
        tensor_dst = tensor_src.to(f'cuda:{dst}', non_blocking=False)
        torch.cuda.synchronize()
        end = time.time()
        times.append(end - start)

    avg_time = sum(times[1:]) / (num_iters - 1)  # 排除首次warmup
    bandwidth = size_gb / avg_time
    print(f"平均耗时: {avg_time*1000:.2f} ms")
    print(f"带宽: {bandwidth:.2f} GB/s")
    print(f"{'='*80}\n")

    # 多卡全互通模式
    print(f"测试多卡全互通带宽（所有显卡之间两两拷贝）:")
    results = []
    for src in range(torch.cuda.device_count()):
        for dst in range(torch.cuda.device_count()):
            if src == dst:
                continue
            tensor_src = torch.randn(size//8, dtype=torch.float32, device=f'cuda:{src}')
            torch.cuda.synchronize()
            times = []
            for _ in range(num_iters):
                torch.cuda.synchronize()
                start = time.time()
                tensor_dst = tensor_src.to(f'cuda:{dst}', non_blocking=False)
                torch.cuda.synchronize()
                end = time.time()
                times.append(end - start)
            avg_time = sum(times[1:]) / (num_iters - 1)
            bandwidth = (size_gb/8) / avg_time
            results.append((src, dst, bandwidth))
            print(f"  GPU{src} → GPU{dst}: {bandwidth:.2f} GB/s (avg {avg_time*1000:.2f} ms)")
    print(f"{'='*80}\n")
    return results

# 只有有多于一张卡时才测试
if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    test_inter_gpu_bandwidth()
else:
    print("本机没有多张GPU，跳过跨卡带宽测试。")



测试跨GPU拷贝速度: 从GPU0 --> GPU1 (1 GB, 10次)
平均耗时: 4.00 ms
带宽: 250.11 GB/s

测试多卡全互通带宽（所有显卡之间两两拷贝）:
  GPU0 → GPU1: 229.94 GB/s (avg 0.54 ms)
  GPU0 → GPU2: 10.39 GB/s (avg 12.03 ms)
  GPU0 → GPU3: 10.40 GB/s (avg 12.02 ms)
  GPU1 → GPU0: 230.90 GB/s (avg 0.54 ms)
  GPU1 → GPU2: 15.64 GB/s (avg 7.99 ms)
  GPU1 → GPU3: 10.31 GB/s (avg 12.12 ms)
  GPU2 → GPU0: 10.06 GB/s (avg 12.42 ms)
  GPU2 → GPU1: 15.74 GB/s (avg 7.94 ms)
  GPU2 → GPU3: 1083.24 GB/s (avg 0.12 ms)
  GPU3 → GPU0: 10.39 GB/s (avg 12.03 ms)
  GPU3 → GPU1: 15.73 GB/s (avg 7.95 ms)
  GPU3 → GPU2: 1507.06 GB/s (avg 0.08 ms)



In [6]:
# 检查GPU拓扑和P2P连接，解释为什么GPU0和GPU1之间快，其他慢
import torch
import subprocess
import os

print("=" * 80)
print("GPU拓扑和P2P连接分析")
print("=" * 80)

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"\n检测到 {num_gpus} 个GPU设备\n")
    
    # 1. 检查PyTorch的P2P访问能力
    print("1. GPU之间的P2P (Peer-to-Peer) 访问能力:")
    print("-" * 80)
    p2p_matrix = {}
    for i in range(num_gpus):
        for j in range(num_gpus):
            if i != j:
                can_access = torch.cuda.can_device_access_peer(i, j)
                p2p_matrix[(i, j)] = can_access
                status = "✓ 支持" if can_access else "✗ 不支持"
                print(f"  GPU{i} → GPU{j}: {status}")
    
    # 2. 尝试使用nvidia-smi查看拓扑（如果可用）
    print("\n2. 使用nvidia-smi查看GPU拓扑:")
    print("-" * 80)
    try:
        # 检查nvidia-smi是否可用
        result = subprocess.run(['nvidia-smi', 'topo', '-m'], 
                               capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print(result.stdout)
        else:
            print("  nvidia-smi topo命令不可用，尝试其他方法...")
    except (subprocess.TimeoutExpired, FileNotFoundError, subprocess.SubprocessError):
        print("  nvidia-smi命令不可用或超时")
    
    # 3. 解析nvidia-smi拓扑信息
    print("\n3. 拓扑连接类型解析:")
    print("-" * 80)
    print("根据nvidia-smi topo输出:")
    print("  • PIX: 通过单个PCIe桥接（同一PCIe域内，较快）")
    print("  • SYS: 通过PCIe + CPU互连（跨NUMA节点，很慢）")
    print("  • NV#: 通过NVLink直连（最快，但您的系统未显示）")
    print()
    
    # 分析NUMA拓扑
    numa_groups = {}
    try:
        result = subprocess.run(['nvidia-smi', 'topo', '-m'], 
                               capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            lines = result.stdout.split('\n')
            for line in lines:
                if 'CPU Affinity' in line or 'NUMA Affinity' in line:
                    # 解析NUMA信息
                    for i, gpu_line in enumerate(lines):
                        if i > 0 and 'GPU' in gpu_line and '\t' in gpu_line:
                            parts = gpu_line.split('\t')
                            if len(parts) >= 6:
                                gpu_id = i - 1
                                numa_affinity = parts[5] if len(parts) > 5 else 'N/A'
                                numa_groups[gpu_id] = numa_affinity
    except:
        pass
    
    # 4. 检查PCIe配置信息
    print("\n4. PCIe配置信息分析:")
    print("-" * 80)
    pcie_info = {}
    try:
        # 使用lspci获取PCIe信息
        result = subprocess.run(['lspci', '-v'], capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            lines = result.stdout.split('\n')
            current_gpu = None
            for line in lines:
                if 'VGA compatible controller' in line or '3D controller' in line:
                    if 'NVIDIA' in line:
                        # 提取PCIe地址
                        pcie_addr = line.split()[0]
                        current_gpu = pcie_addr
                        pcie_info[current_gpu] = {'addr': pcie_addr, 'details': []}
                elif current_gpu and ('LnkSta' in line or 'LnkCap' in line or 'PCIe' in line):
                    pcie_info[current_gpu]['details'].append(line.strip())
    except:
        pass
    
    # 使用nvidia-smi查询PCIe信息
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=index,pci.bus_id,pcie.link.gen.current,pcie.link.width.current', 
                                '--format=csv,noheader'], capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print("PCIe链路信息:")
            for line in result.stdout.strip().split('\n'):
                parts = line.split(', ')
                if len(parts) >= 4:
                    gpu_idx = parts[0].strip()
                    pci_bus = parts[1].strip()
                    pcie_gen = parts[2].strip()
                    pcie_width = parts[3].strip()
                    print(f"  GPU{gpu_idx}: PCIe总线={pci_bus}, 版本=Gen{pcie_gen}, 宽度=x{pcie_width}")
                    pcie_info[f'gpu{gpu_idx}'] = {
                        'gen': pcie_gen,
                        'width': pcie_width,
                        'bus': pci_bus
                    }
    except:
        print("  无法获取PCIe详细信息")
    
    # 5. 详细分析（无NVLink情况）
    print("\n5. 无NVLink情况下的速度差异分析:")
    print("-" * 80)
    print("您的系统拓扑结构:")
    print("  • GPU0和GPU1: 同一NUMA节点（节点0），有P2P支持")
    print("  • GPU2和GPU3: 同一NUMA节点（节点1），有P2P支持")
    print("  • 系统无NVLink连接")
    print()
    print("速度差异的根本原因（无NVLink）:")
    print()
    
    # 检查GPU0和GPU1的P2P
    if p2p_matrix.get((0, 1), False) and p2p_matrix.get((1, 0), False):
        print("  ✓ GPU0 ↔ GPU1: 速度快（~1000+ GB/s）")
        print("    → 有P2P支持，通过PCIe P2P通信")
        print("    → 可能的原因:")
        print("       1. 同一PCIe根复合体（Root Complex）下")
        print("       2. 通过PCIe交换机直接连接（PIX连接）")
        print("       3. PCIe 4.0 x16配置，理论带宽64 GB/s（双向128 GB/s）")
        print("       4. 数据直接在PCIe域内传输，无需经过CPU内存")
        print("    → 虽然无NVLink，但PCIe P2P仍然可以实现较高速度")
    else:
        print("  ⚠ GPU0和GPU1之间没有P2P支持")
    
    print()
    if p2p_matrix.get((2, 3), False) and p2p_matrix.get((3, 2), False):
        print("  ⚠ GPU2 ↔ GPU3: 速度较慢（即使有P2P支持）")
        print("    → 虽然有P2P支持，但速度仍然较慢，可能的原因:")
        print("       1. 不同的PCIe根复合体配置")
        print("       2. PCIe版本较低（如PCIe 3.0 vs 4.0）")
        print("       3. PCIe通道数较少（如x8 vs x16）")
        print("       4. 需要通过额外的PCIe桥接芯片，增加延迟")
        print("       5. PCIe插槽类型不同（如x16物理但x8电气）")
        print("    → 检查PCIe配置:")
        if 'gpu2' in pcie_info and 'gpu3' in pcie_info:
            gpu2_gen = pcie_info.get('gpu2', {}).get('gen', 'N/A')
            gpu2_width = pcie_info.get('gpu2', {}).get('width', 'N/A')
            gpu3_gen = pcie_info.get('gpu3', {}).get('gen', 'N/A')
            gpu3_width = pcie_info.get('gpu3', {}).get('width', 'N/A')
            print(f"       GPU2: PCIe Gen{gpu2_gen} x{gpu2_width}")
            print(f"       GPU3: PCIe Gen{gpu3_gen} x{gpu3_width}")
        if 'gpu0' in pcie_info and 'gpu1' in pcie_info:
            gpu0_gen = pcie_info.get('gpu0', {}).get('gen', 'N/A')
            gpu0_width = pcie_info.get('gpu0', {}).get('width', 'N/A')
            gpu1_gen = pcie_info.get('gpu1', {}).get('gen', 'N/A')
            gpu1_width = pcie_info.get('gpu1', {}).get('width', 'N/A')
            print(f"       GPU0: PCIe Gen{gpu0_gen} x{gpu0_width}")
            print(f"       GPU1: PCIe Gen{gpu1_gen} x{gpu1_width}")
            print()
            print("    → 对比分析:")
            if gpu0_gen != 'N/A' and gpu2_gen != 'N/A':
                if gpu0_gen > gpu2_gen:
                    print(f"       GPU0/1使用PCIe Gen{gpu0_gen}，而GPU2/3使用PCIe Gen{gpu2_gen}")
                    print(f"       PCIe Gen{gpu0_gen}带宽是Gen{gpu2_gen}的2倍")
                elif gpu0_width != 'N/A' and gpu2_width != 'N/A':
                    if int(gpu0_width.replace('x', '')) > int(gpu2_width.replace('x', '')):
                        print(f"       GPU0/1使用x{gpu0_width}，而GPU2/3使用x{gpu2_width}")
                        print(f"       更多通道数意味着更高带宽")
    else:
        print("  ✗ GPU2 ↔ GPU3: 无P2P支持")
        print("    → 必须通过CPU内存中转，速度很慢")
    
    print()
    print("  ⚠ GPU0/1 ↔ GPU2/3: 1-15 GB/s（跨NUMA节点通信）")
    print("    → 无P2P支持，必须通过CPU和PCIe")
    print("    → 需要经过以下路径:")
    print("       1. GPU0 → PCIe → CPU0内存 (NUMA节点0)")
    print("       2. CPU0内存 → QPI/UPI互连 → CPU1内存 (NUMA节点1)")
    print("       3. CPU1内存 → PCIe → GPU2")
    print("    → 这个路径涉及多次数据拷贝和CPU参与，导致:")
    print("       • 高延迟（多次内存拷贝）")
    print("       • 低带宽（受CPU互连带宽限制，通常<20 GB/s）")
    print("    → 这是典型的NUMA架构限制")
    
    # 6. 检查GPU属性
    print("\n6. GPU设备属性:")
    print("-" * 80)
    for i in range(num_gpus):
        props = torch.cuda.get_device_properties(i)
        print(f"  GPU{i}:")
        print(f"    名称: {props.name}")
        print(f"    PCI总线ID: {props.pci_bus_id if hasattr(props, 'pci_bus_id') else 'N/A'}")
        print(f"    PCI设备ID: {props.pci_device_id if hasattr(props, 'pci_device_id') else 'N/A'}")
        print(f"    多处理器数量: {props.multi_processor_count}")
    
    # 7. 优化建议（无NVLink情况）
    print("\n7. 针对无NVLink系统的优化建议:")
    print("-" * 80)
    print("基于NUMA拓扑的优化策略:")
    print()
    print("  1. 数据并行训练:")
    print("     • 将模型副本放在同一NUMA节点内（GPU0+GPU1 或 GPU2+GPU3）")
    print("     • 避免跨NUMA节点的频繁通信")
    print("     • 使用DataParallel时，设置device_ids=[0,1]或[2,3]")
    print()
    print("  2. 模型并行训练:")
    print("     • 如果模型很大，将不同层放在同一NUMA节点的GPU上")
    print("     • 例如：前几层在GPU0，后几层在GPU1（都在节点0）")
    print("     • 避免将模型分割到跨NUMA节点的GPU上")
    print()
    print("  3. 混合精度和通信优化:")
    print("     • 使用梯度压缩（如FP16）减少跨NUMA通信的数据量")
    print("     • 使用梯度累积，减少通信频率")
    print("     • NCCL会自动选择最优路径，但跨NUMA仍然很慢")
    print()
    print("  4. 进程绑定:")
    print("     • 使用numactl将进程绑定到特定NUMA节点")
    print("     • 例如：numactl --cpunodebind=0 --membind=0 python train.py")
    print("     • 这可以避免跨NUMA节点的内存访问")
    print()
    print("  5. 如果必须使用所有4个GPU:")
    print("     • 考虑使用2个独立的训练进程（每个使用2个GPU）")
    print("     • 或者接受跨NUMA通信的性能损失")
    print("     • 使用更大的batch size和更少的通信频率来补偿")
    
    # 8. PCIe P2P性能说明
    print("\n8. PCIe P2P性能说明:")
    print("-" * 80)
    print("PCIe P2P（Peer-to-Peer）通信特点:")
    print("  • 允许GPU之间直接通过PCIe通信，无需经过CPU内存")
    print("  • 性能取决于PCIe配置:")
    print("     - PCIe 4.0 x16: 理论单向32 GB/s，双向64 GB/s")
    print("     - PCIe 3.0 x16: 理论单向16 GB/s，双向32 GB/s")
    print("     - PCIe 4.0 x8:  理论单向16 GB/s，双向32 GB/s")
    print("     - PCIe 3.0 x8:  理论单向8 GB/s，双向16 GB/s")
    print("  • 实际测试速度可能接近理论值（如果配置正确）")
    print("  • GPU0 ↔ GPU1速度快，说明PCIe P2P配置良好")
    print("  • GPU2 ↔ GPU3速度慢，可能是PCIe配置不同或受限")
    print()
    print("检查PCIe配置差异的方法:")
    print("  • 运行: nvidia-smi --query-gpu=index,pcie.link.gen.current,pcie.link.width.current --format=csv")
    print("  • 检查BIOS中的PCIe配置")
    print("  • 确认所有GPU插槽都配置为x16（如果主板支持）")
    try:
        # 尝试使用nvidia-ml-py获取更详细的信息
        try:
            import pynvml
            pynvml.nvmlInit()
            for i in range(num_gpus):
                handle = pynvml.nvmlDeviceGetHandleByIndex(i)
                try:
                    # 获取NVLink信息（如果可用）
                    nvlink_version = pynvml.nvmlDeviceGetNvLinkVersion(handle, 0)
                    print(f"  GPU{i} NVLink版本: {nvlink_version}")
                except:
                    pass
            pynvml.nvmlShutdown()
        except ImportError:
            print("  提示: 安装pynvml可以获取更详细的NVLink信息")
            print("        命令: pip install nvidia-ml-py3")
        except Exception as e:
            print(f"  无法获取NVLink信息: {e}")
    except:
        pass

else:
    print("没有可用的GPU")

print("\n" + "=" * 80)

GPU拓扑和P2P连接分析

检测到 4 个GPU设备

1. GPU之间的P2P (Peer-to-Peer) 访问能力:
--------------------------------------------------------------------------------
  GPU0 → GPU1: ✓ 支持
  GPU0 → GPU2: ✗ 不支持
  GPU0 → GPU3: ✗ 不支持
  GPU1 → GPU0: ✓ 支持
  GPU1 → GPU2: ✗ 不支持
  GPU1 → GPU3: ✗ 不支持
  GPU2 → GPU0: ✗ 不支持
  GPU2 → GPU1: ✗ 不支持
  GPU2 → GPU3: ✓ 支持
  GPU3 → GPU0: ✗ 不支持
  GPU3 → GPU1: ✗ 不支持
  GPU3 → GPU2: ✓ 支持

2. 使用nvidia-smi查看GPU拓扑:
--------------------------------------------------------------------------------
	GPU0	GPU1	GPU2	GPU3	CPU Affinity	NUMA Affinity	GPU NUMA ID
GPU0	 X 	NV12	SYS	SYS	0-21,44-65	0		N/A
GPU1	NV12	 X 	SYS	SYS	0-21,44-65	0		N/A
GPU2	SYS	SYS	 X 	NV12	22-43,66-87	1		N/A
GPU3	SYS	SYS	NV12	 X 	22-43,66-87	1		N/A

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well 